In [1]:
from shapely.geometry import Point
from collections import Counter
import seaborn as sns
import datetime as datetime
from datetime import date
import io
import re
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

In [2]:
mma_data = pd.read_csv('UFC.csv')

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (144,145) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
mma_data

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Bantamweight,0.000,0.0,...,Orthodox,170.18,177.80,135.0,31.0,27.0,1.04,1.03,7.62,5.08
1,Trevin Giles,Roman Dolidze,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Middleweight,0.500,0.0,...,Orthodox,182.88,187.96,185.0,32.0,28.0,1.03,1.03,5.08,5.08
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,3/20/2021,"Las Vegas, Nevada, USA",Red,False,Heavyweight,NaN,NaN,...,Southpaw,187.96,190.50,264.0,32.0,28.0,1.01,1.01,2.54,2.54
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenStrawweight,NaN,NaN,...,Switch,160.02,160.02,115.0,28.0,25.0,1.0,1.02,0.00,2.54
4,Marion Reneau,Macy Chiasson,Mark Smith,3/20/2021,"Las Vegas, Nevada, USA",Blue,False,WomenBantamweight,0.125,0.0,...,Orthodox,167.64,172.72,135.0,29.0,43.0,1.03,1.01,5.08,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,190.50,NaN,260.0,NaN,NaN,0,0,-190.50,-172.72
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Southpaw,177.80,NaN,170.0,NaN,NaN,0,0,-177.80,-187.96
6009,Johnny Rhodes,David Levicki,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,182.88,NaN,210.0,NaN,NaN,0,0,-182.88,-195.58
6010,Patrick Smith,Ray Wizard,John McCarthy,3/11/1994,"Denver, Colorado, USA",Red,False,OpenWeight,NaN,NaN,...,Orthodox,187.96,NaN,225.0,NaN,30.0,0,#DIV/0!,-187.96,0.00


In [4]:
mma_data[mma_data['R_fighter'] == 'Dustin Poirier']

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
85,Dustin Poirier,Conor McGregor,Herb Dean,1/23/2021,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Lightweight,1.093262,0.000000,...,Southpaw,175.26,182.88,155.0,32.0,32.0,1.04,1.07,7.62,12.70
395,Dustin Poirier,Dan Hooker,Herb Dean,6/27/2020,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.457275,0.062500,...,Southpaw,175.26,182.88,155.0,30.0,31.0,1.04,1.04,7.62,7.62
1437,Dustin Poirier,Justin Gaethje,Herb Dean,4/14/2018,"Glendale, Arizona, USA",Red,False,Lightweight,0.000000,0.500000,...,Southpaw,175.26,182.88,155.0,29.0,29.0,1.04,0.99,7.62,-2.54
1650,Dustin Poirier,Anthony Pettis,Keith Peterson,11/11/2017,"Norfolk, Virginia, USA",Red,False,Lightweight,0.002930,0.250000,...,Southpaw,175.26,182.88,155.0,30.0,28.0,1.04,1.03,7.62,5.08
1976,Dustin Poirier,Jim Miller,Todd Ronald Anderson,2/11/2017,"Brooklyn, New York, USA",Red,False,Lightweight,0.000017,0.003906,...,Southpaw,175.26,182.88,155.0,33.0,28.0,1.04,1.04,7.62,7.62
2176,Dustin Poirier,Michael Johnson,Dan Miragliotta,9/17/2016,"Hidalgo, Texas, USA",Blue,False,Lightweight,0.073486,0.001953,...,Southpaw,175.26,182.88,155.0,30.0,27.0,1.04,1.04,7.62,7.62
2333,Dustin Poirier,Bobby Green,Jason Herzog,6/4/2016,"Los Angeles, California, USA",Red,False,Lightweight,0.062500,0.000000,...,Southpaw,175.26,182.88,155.0,29.0,27.0,1.04,1.01,7.62,2.54
2509,Dustin Poirier,Joe Duffy,Chris Tognoni,1/2/2016,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.500000,0.000000,...,Southpaw,175.26,182.88,155.0,27.0,26.0,1.04,1.04,7.62,7.62
2783,Dustin Poirier,Yancy Medeiros,John McCarthy,6/6/2015,"New Orleans, Louisiana, USA",Red,False,Lightweight,0.562500,0.000000,...,Southpaw,175.26,182.88,155.0,27.0,26.0,1.04,1.07,7.62,12.70
2879,Dustin Poirier,Diego Ferreira,Todd McGovern,4/4/2015,"Fairfax, Virginia, USA",Red,False,Lightweight,0.250000,0.000000,...,Southpaw,175.26,182.88,155.0,30.0,26.0,1.04,1.07,7.62,12.70


In [5]:
mma_data[mma_data['B_fighter'] == 'Dustin Poirier']

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age,R_index_ape,B_index_ape,R_index_ape2,B_index_ape2
737,Khabib Nurmagomedov,Dustin Poirier,Marc Goddard,9/7/2019,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,True,Lightweight,0.260986,0.008057,...,Orthodox,177.80,177.80,155.0,30.0,30.0,1.0,1.04,0.00,7.62
951,Max Holloway,Dustin Poirier,Blake Grice,4/13/2019,"Atlanta, Georgia, USA",Blue,True,Lightweight,0.521973,0.016113,...,Orthodox,180.34,175.26,145.0,30.0,27.0,0.97,1.04,-5.08,7.62
1292,Eddie Alvarez,Dustin Poirier,Marc Goddard,7/28/2018,"Calgary, Alberta, Canada",Blue,False,Lightweight,0.043945,0.032227,...,Orthodox,175.26,175.26,155.0,29.0,34.0,1.0,1.04,0.00,7.62
1880,Eddie Alvarez,Dustin Poirier,Herb Dean,5/13/2017,"Dallas, Texas, USA",Draw,False,Lightweight,0.351562,0.257812,...,Orthodox,175.26,175.26,155.0,28.0,33.0,1.0,1.04,0.00,7.62
3629,Erik Koch,Dustin Poirier,Rob Hinds,8/31/2013,"Milwaukee, Wisconsin, USA",Blue,False,Featherweight,0.000000,0.000000,...,Southpaw,177.80,180.34,170.0,24.0,24.0,1.01,1.04,2.54,7.62
3829,Cub Swanson,Dustin Poirier,Kevin Mulhall,2/16/2013,"London, England, United Kingdom",Red,False,Featherweight,0.000000,0.000000,...,Orthodox,172.72,177.80,145.0,24.0,29.0,1.03,1.04,5.08,7.62
4101,Chan Sung Jung,Dustin Poirier,Dan Miragliotta,5/15/2012,"Fairfax, Virginia, USA",Red,False,Featherweight,0.000000,0.000000,...,Orthodox,170.18,182.88,145.0,23.0,25.0,1.07,1.04,12.70,7.62


In [6]:
mma_data['date'] = pd.to_datetime(mma_data['date'])
ufc_unified_rules = mma_data[~(mma_data['date'] < '2000-01-01')]

In [7]:
r_first_appearance = ufc_unified_rules.groupby('R_fighter')['date'].min().reset_index()

In [8]:
r_last_appearance = ufc_unified_rules.groupby('R_fighter')['date'].max().reset_index()

In [9]:
r_time_active = r_last_appearance['date'] - r_first_appearance['date']

In [10]:
b_first_appearance = ufc_unified_rules.groupby('B_fighter')['date'].min().reset_index()

In [11]:
b_last_appearance = ufc_unified_rules.groupby('B_fighter')['date'].max().reset_index()

In [12]:
b_time_active = b_last_appearance['date'] - b_first_appearance['date']

In [13]:
ufc_unified_rules.rename(columns = {'date': 'fight date'}, inplace = True)

/opt/anaconda3/envs/geospatial/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [14]:
ufc_unified_rules = pd.concat([ufc_unified_rules, r_time_active.reindex(ufc_unified_rules.index)], axis = 1)

In [15]:
ufc_unified_rules.rename(columns = {'date': 'r time active'}, inplace = True)

In [16]:
ufc_unified_rules = pd.concat([ufc_unified_rules, b_time_active.reindex(ufc_unified_rules.index)], axis = 1)

In [17]:
ufc_unified_rules.rename(columns = {'date': 'b time active'}, inplace = True)

In [18]:
title_data = ufc_unified_rules[['R_fighter', 'B_fighter', 'title_bout', 'fight date', 'weight_class', 'Winner']]

In [19]:
win_loss_draw = mma_data[['R_fighter', 'B_fighter','R_wins', 'R_draw', 'R_losses', 'B_wins', 'B_draw', 'B_losses']]

In [20]:
r_fighter_stats = ufc_unified_rules.drop(columns = ['B_fighter', 'B_avg_KD', 'B_avg_opp_KD', 'B_avg_SIG_STR_pct', 
                                         'B_avg_opp_SIG_STR_pct', 'B_avg_TD_pct', 'B_avg_opp_TD_pct', 
                                         'B_avg_SUB_ATT', 'B_avg_opp_SUB_ATT', 'B_avg_REV', 'B_avg_opp_REV', 
                                         'B_avg_SIG_STR_att', 'B_avg_SIG_STR_landed', 'B_avg_opp_SIG_STR_att', 
                                         'B_avg_opp_SIG_STR_landed', 'B_avg_TOTAL_STR_att', 'B_avg_TOTAL_STR_landed', 
                                         'B_avg_opp_TOTAL_STR_att', 'B_avg_opp_TOTAL_STR_landed', 'B_avg_TD_att', 
                                         'B_avg_TD_landed', 'B_avg_opp_TD_att', 'B_avg_opp_TD_landed', 'B_avg_HEAD_att', 
                                         'B_avg_HEAD_landed', 'B_avg_opp_HEAD_att', 'B_avg_opp_HEAD_landed', 
                                         'B_avg_BODY_att', 'B_avg_BODY_landed', 'B_avg_opp_BODY_att', 
                                         'B_avg_opp_BODY_landed', 'B_avg_LEG_att', 'B_avg_LEG_landed', 
                                         'B_avg_opp_LEG_att', 'B_avg_opp_LEG_landed', 'B_avg_DISTANCE_att', 
                                         'B_avg_DISTANCE_landed', 'B_avg_opp_DISTANCE_att', 'B_avg_opp_DISTANCE_landed', 
                                         'B_avg_CLINCH_att', 'B_avg_CLINCH_landed', 'B_avg_opp_CLINCH_att', 
                                         'B_avg_opp_CLINCH_landed', 'B_avg_GROUND_att', 'B_avg_GROUND_landed', 
                                         'B_avg_opp_GROUND_att', 'B_avg_opp_GROUND_landed', 'B_avg_CTRL_time(seconds)',
                                         'B_avg_opp_CTRL_time(seconds)', 'B_total_time_fought(seconds)', 
                                         'B_total_rounds_fought', 'B_total_title_bouts', 'B_current_win_streak', 
                                         'B_current_lose_streak', 'B_longest_win_streak', 'B_wins', 'B_losses', 
                                         'B_draw', 'B_win_by_Decision_Majority', 'B_win_by_Decision_Split', 
                                         'B_win_by_Decision_Unanimous', 'B_win_by_KO/TKO', 'B_win_by_Submission', 
                                         'B_win_by_TKO_Doctor_Stoppage', 'B_Stance', 'B_Height_cms', 'B_Reach_cms', 
                                         'B_Weight_lbs', 'B_age', 'B_index_ape', 'B_index_ape2', 'b time active'])

In [21]:
r_fighter_stats[['R_fighter', 'R_wins', 'R_draw', 'R_losses']]

,R_fighter,R_wins,R_draw,R_losses
0,Adrian Yanez,1,0,0
1,Trevin Giles,4,0,2
2,Tai Tuivasa,4,0,3
3,Cheyanne Buys,0,0,0
4,Marion Reneau,5,0,6
...,...,...,...,...
5804,Scott Adams,0,0,0
5805,Jens Pulver,0,0,1
5806,Tedd Williams,0,0,0
5807,Bob Cook,0,0,0


In [22]:
r_total_fights = r_fighter_stats["R_wins"] + r_fighter_stats["R_losses"] + r_fighter_stats["R_draw"]

In [23]:
r_fighter_stats["total_fights"] = r_total_fights

In [24]:
r_win_pct = ((r_fighter_stats["R_wins"] / r_fighter_stats["total_fights"]) * 100)

In [25]:
r_fighter_stats["win_pct"] = r_win_pct

In [26]:
b_fighter_stats = ufc_unified_rules.drop(columns = ['R_fighter', 'R_avg_KD', 'R_avg_opp_KD', 'R_avg_SIG_STR_pct', 
                                         'R_avg_opp_SIG_STR_pct', 'R_avg_TD_pct', 'R_avg_opp_TD_pct', 
                                         'R_avg_SUB_ATT', 'R_avg_opp_SUB_ATT', 'R_avg_REV', 'R_avg_opp_REV', 
                                         'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_opp_SIG_STR_att', 
                                         'R_avg_opp_SIG_STR_landed', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 
                                         'R_avg_opp_TOTAL_STR_att', 'R_avg_opp_TOTAL_STR_landed', 'R_avg_TD_att', 
                                         'R_avg_TD_landed', 'R_avg_opp_TD_att', 'R_avg_opp_TD_landed', 'R_avg_HEAD_att', 
                                         'R_avg_HEAD_landed', 'R_avg_opp_HEAD_att', 'R_avg_opp_HEAD_landed', 
                                         'R_avg_BODY_att', 'R_avg_BODY_landed', 'R_avg_opp_BODY_att', 
                                         'R_avg_opp_BODY_landed', 'R_avg_LEG_att', 'R_avg_LEG_landed', 
                                         'R_avg_opp_LEG_att', 'R_avg_opp_LEG_landed', 'R_avg_DISTANCE_att', 
                                         'R_avg_DISTANCE_landed', 'R_avg_opp_DISTANCE_att', 'R_avg_opp_DISTANCE_landed', 
                                         'R_avg_CLINCH_att', 'R_avg_CLINCH_landed', 'R_avg_opp_CLINCH_att', 
                                         'R_avg_opp_CLINCH_landed', 'R_avg_GROUND_att', 'R_avg_GROUND_landed', 
                                         'R_avg_opp_GROUND_att', 'R_avg_opp_GROUND_landed', 'R_avg_CTRL_time(seconds)',
                                         'R_avg_opp_CTRL_time(seconds)', 'R_total_time_fought(seconds)', 
                                         'R_total_rounds_fought', 'R_total_title_bouts', 'R_current_win_streak', 
                                         'R_current_lose_streak', 'R_longest_win_streak', 'R_wins', 'R_losses', 
                                         'R_draw', 'R_win_by_Decision_Majority', 'R_win_by_Decision_Split', 
                                         'R_win_by_Decision_Unanimous', 'R_win_by_KO/TKO', 'R_win_by_Submission', 
                                         'R_win_by_TKO_Doctor_Stoppage', 'R_Stance', 'R_Height_cms', 'R_Reach_cms', 
                                         'R_Weight_lbs', 'R_age', 'R_index_ape', 'R_index_ape2', 'r time active'])

In [27]:
b_fighter_stats[['B_wins', 'B_draw', 'B_losses']]

,B_wins,B_draw,B_losses
0,1,0,1
1,2,0,0
2,0,0,0
3,0,0,0
4,4,0,1
...,...,...,...
5804,0,0,0
5805,0,0,0
5806,0,0,1
5807,0,0,0


In [28]:
b_total_fights = b_fighter_stats["B_wins"] + b_fighter_stats["B_losses"] + b_fighter_stats["B_draw"]

In [29]:
b_fighter_stats["total_fights"] = b_total_fights

In [30]:
b_win_pct = ((b_fighter_stats["B_wins"] / b_fighter_stats["total_fights"]) * 100)

In [31]:
b_fighter_stats["win_pct"] = b_win_pct

In [32]:
r_fighter_stats[r_fighter_stats['R_fighter'] == 'Dustin Poirier']

,R_fighter,Referee,fight date,location,Winner,title_bout,weight_class,R_avg_KD,R_avg_opp_KD,R_avg_SIG_STR_pct,...,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,R_index_ape,R_index_ape2,r time active,total_fights,win_pct
85,Dustin Poirier,Herb Dean,2021-01-23,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,False,Lightweight,0.065247,0.002014,0.522531,...,Southpaw,175.26,182.88,155.0,32.0,1.04,7.62,1345 days,24,75.000000
395,Dustin Poirier,Herb Dean,2020-06-27,"Las Vegas, Nevada, USA",Red,False,Lightweight,0.130493,0.004028,0.385061,...,Southpaw,175.26,182.88,155.0,31.0,1.04,7.62,607 days,23,73.913043
1437,Dustin Poirier,Herb Dean,2018-04-14,"Glendale, Arizona, USA",Red,False,Lightweight,0.087891,0.064453,0.510980,...,Southpaw,175.26,182.88,155.0,29.0,1.04,7.62,0 days,19,73.684211
1650,Dustin Poirier,Keith Peterson,2017-11-11,"Norfolk, Virginia, USA",Red,False,Lightweight,0.175781,0.128906,0.491959,...,Southpaw,175.26,182.88,155.0,28.0,1.04,7.62,NaT,18,72.222222
1976,Dustin Poirier,Todd Ronald Anderson,2017-02-11,"Brooklyn, New York, USA",Red,False,Lightweight,0.703125,0.515625,0.527837,...,Southpaw,175.26,182.88,155.0,28.0,1.04,7.62,NaT,16,75.000000
2176,Dustin Poirier,Dan Miragliotta,2016-09-17,"Hidalgo, Texas, USA",Blue,False,Lightweight,1.406250,0.031250,0.555674,...,Southpaw,175.26,182.88,155.0,27.0,1.04,7.62,NaT,15,80.000000
2333,Dustin Poirier,Jason Herzog,2016-06-04,"Los Angeles, California, USA",Red,False,Lightweight,0.812500,0.062500,0.571348,...,Southpaw,175.26,182.88,155.0,27.0,1.04,7.62,NaT,14,78.571429
2509,Dustin Poirier,Chris Tognoni,2016-01-02,"Las Vegas, Nevada, USA",Red,False,Lightweight,1.625000,0.125000,0.622695,...,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT,13,76.923077
2783,Dustin Poirier,John McCarthy,2015-06-06,"New Orleans, Louisiana, USA",Red,False,Lightweight,1.250000,0.250000,0.495391,...,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT,12,75.000000
2879,Dustin Poirier,Todd McGovern,2015-04-04,"Fairfax, Virginia, USA",Red,False,Lightweight,0.500000,0.500000,0.510781,...,Southpaw,175.26,182.88,155.0,26.0,1.04,7.62,NaT,11,72.727273


In [33]:
b_fighter_stats[b_fighter_stats['B_fighter'] == 'Dustin Poirier']

,B_fighter,Referee,fight date,location,Winner,title_bout,weight_class,B_avg_KD,B_avg_opp_KD,B_avg_SIG_STR_pct,...,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,B_age,B_index_ape,B_index_ape2,b time active,total_fights,win_pct
737,Dustin Poirier,Marc Goddard,2019-09-07,"Abu Dhabi, Abu Dhabi, United Arab Emirates",Red,True,Lightweight,0.260986,0.008057,0.470122,...,Southpaw,175.26,182.88,155.0,30.0,1.04,7.62,0 days,22,77.272727
951,Dustin Poirier,Blake Grice,2019-04-13,"Atlanta, Georgia, USA",Blue,True,Lightweight,0.521973,0.016113,0.480245,...,Southpaw,175.26,182.88,155.0,30.0,1.04,7.62,0 days,21,76.190476
1292,Dustin Poirier,Marc Goddard,2018-07-28,"Calgary, Alberta, Canada",Blue,False,Lightweight,0.043945,0.032227,0.500490,...,Southpaw,175.26,182.88,155.0,29.0,1.04,7.62,2912 days,20,75.000000
1880,Dustin Poirier,Herb Dean,2017-05-13,"Dallas, Texas, USA",Draw,False,Lightweight,0.351562,0.257812,0.483918,...,Southpaw,175.26,182.88,155.0,28.0,1.04,7.62,NaT,17,76.470588
3629,Dustin Poirier,Rob Hinds,2013-08-31,"Milwaukee, Wisconsin, USA",Blue,False,Featherweight,0.000000,0.000000,0.432500,...,Southpaw,175.26,182.88,155.0,24.0,1.04,7.62,NaT,7,71.428571
3829,Dustin Poirier,Kevin Mulhall,2013-02-16,"London, England, United Kingdom",Red,False,Featherweight,0.000000,0.000000,0.455000,...,Southpaw,175.26,182.88,155.0,24.0,1.04,7.62,NaT,6,83.333333
4101,Dustin Poirier,Dan Miragliotta,2012-05-15,"Fairfax, Virginia, USA",Red,False,Featherweight,0.000000,0.000000,0.440000,...,Southpaw,175.26,182.88,155.0,23.0,1.04,7.62,NaT,4,100.000000


In [34]:
active_fighters = pd.read_csv('fighter_df.csv')

In [35]:
total_fighter_weightclass = active_fighters[["WeightClass"]]

In [36]:
total_fighter_weightclass.value_counts()

WeightClass          
Lightweight              155
Welterweight             151
Featherweight            146
Bantamweight             134
Middleweight             111
Light Heavyweight         89
Heavyweight               87
Women's Strawweight       67
Women's Flyweight         57
Flyweight                 55
Women's Bantamweight      41
Women's Featherweight     10
Catch Weight               1
Catchweight                1
dtype: int64

In [37]:
active_fighters = active_fighters.drop(columns = ['CareerStats.FighterId',
                                                  'CareerStats.FirstName', 
                                                  'CareerStats.LastName'])

In [38]:
active_fighters['BirthDate'] = pd.to_datetime(active_fighters['BirthDate'])

In [39]:
print(active_fighters[['Nickname']])

               Nickname
0           Black Eagle
1               Gunz Up
2        Handz of Steel
3     Remember the Name
4            The Menace
...                 ...
1158                NaN
1159               C.J.
1160   The Snow Leopard
1161                NaN
1162    Chidi Bang Bang

[1163 rows x 1 columns]


In [40]:
nickname_list = active_fighters.index.tolist()

In [41]:
fighter_nickname = active_fighters["Nickname"].tolist()

In [42]:
total_fighter_nickname_count = Counter(fighter_nickname)

In [43]:
total_fighter_nickname_count = total_fighter_nickname_count.most_common()

In [44]:
total_fighter_nickname_count

[(nan, 347),
 ('The Spartan', 7),
 ('The Great', 4),
 ('Pitbull', 4),
 ('Danger', 4),
 ('The Sniper', 4),
 ('The Predator', 3),
 ('El Toro', 3),
 ('Bullet', 3),
 ('Wolverine', 3),
 ('The Beast', 3),
 ('The Menace', 2),
 ('Cowboy', 2),
 ('The Kid', 2),
 ('Pretty Boy', 2),
 ('Cyborg', 2),
 ('Junior', 2),
 ('Magic', 2),
 ('Dragon', 2),
 ('The Tarantula', 2),
 ('Golden Boy', 2),
 ('Hot Sauce', 2),
 ('Fury', 2),
 ('Bam Bam', 2),
 ('The Savage', 2),
 ('Kid Dynamite', 2),
 ('The Body Snatcher', 2),
 ('The Barbarian', 2),
 ('Lights Out', 2),
 ('Rocky', 2),
 ('Showtime', 2),
 ('El Guapo', 2),
 ('The Future', 2),
 ('Razor', 2),
 ('Super', 2),
 ('The Warrior', 2),
 ('La Pantera', 2),
 ('The Demolition Man', 2),
 ('The Monster', 2),
 ('The Machine', 2),
 ('Superman', 2),
 ('Hurricane', 2),
 ('The Prodigy', 2),
 ('The Lone Wolf', 2),
 ('The Hurricane', 2),
 ('Triple Threat', 2),
 ('The Bully', 2),
 ('Mr. Highlight', 2),
 ('Flash', 2),
 ('El Nino', 2),
 ('All In', 2),
 ('Lucky', 2),
 ('Grizzly', 2),

In [45]:
total_nickname_count = pd.DataFrame(total_fighter_nickname_count, columns = ['nickname', 'count'])
print(total_fighter_nickname_count)

[(nan, 347), ('The Spartan', 7), ('The Great', 4), ('Pitbull', 4), ('Danger', 4), ('The Sniper', 4), ('The Predator', 3), ('El Toro', 3), ('Bullet', 3), ('Wolverine', 3), ('The Beast', 3), ('The Menace', 2), ('Cowboy', 2), ('The Kid', 2), ('Pretty Boy', 2), ('Cyborg', 2), ('Junior', 2), ('Magic', 2), ('Dragon', 2), ('The Tarantula', 2), ('Golden Boy', 2), ('Hot Sauce', 2), ('Fury', 2), ('Bam Bam', 2), ('The Savage', 2), ('Kid Dynamite', 2), ('The Body Snatcher', 2), ('The Barbarian', 2), ('Lights Out', 2), ('Rocky', 2), ('Showtime', 2), ('El Guapo', 2), ('The Future', 2), ('Razor', 2), ('Super', 2), ('The Warrior', 2), ('La Pantera', 2), ('The Demolition Man', 2), ('The Monster', 2), ('The Machine', 2), ('Superman', 2), ('Hurricane', 2), ('The Prodigy', 2), ('The Lone Wolf', 2), ('The Hurricane', 2), ('Triple Threat', 2), ('The Bully', 2), ('Mr. Highlight', 2), ('Flash', 2), ('El Nino', 2), ('All In', 2), ('Lucky', 2), ('Grizzly', 2), ('Soldier of God', 2), ('Thunder', 2), ('The Leech'

In [46]:
total_nickname_count.head(10)

,nickname,count
0,NaN,347
1,The Spartan,7
2,The Great,4
3,Pitbull,4
4,Danger,4
5,The Sniper,4
6,The Predator,3
7,El Toro,3
8,Bullet,3
9,Wolverine,3


In [47]:
total_nickname_count['count'].dtype

dtype('int64')

In [48]:
total_nickname_count['nickname'].dtype

dtype('O')

In [49]:
now = pd.Timestamp('now')
active_fighters['BirthDate'] = pd.to_datetime(active_fighters['BirthDate'], format = '%m%d%y')
active_fighters['BirthDate'] = active_fighters['BirthDate'].where(active_fighters['BirthDate'] < now, active_fighters['BirthDate'] -  np.timedelta64(100, 'Y'))
active_fighters['age'] = (now - active_fighters['BirthDate']).astype('<m8[Y]')

In [50]:
active_fighters["age"].mean()

31.762757385854968

In [51]:
active_fighters.sort_values(by = 'age', ascending = False)

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats,age
687,140000690,Mark,Hunt,Super Samoan,Heavyweight,1974-03-23,70.0,265.0,72.0,13.0,...,0.0,6.7,80.1,1.2,0.5,53.2,53.2,16.0,NaN,47.0
703,140000706,Mike,Pyle,Quicksand,Welterweight,1975-09-18,73.0,170.0,74.0,27.0,...,0.0,6.5,70.9,4.3,3.7,18.4,18.4,14.7,NaN,46.0
72,140000073,Anderson,Silva,The Spider,Middleweight,1975-04-14,74.0,185.0,77.0,34.0,...,0.0,7.3,95.0,1.2,1.9,63.8,63.8,26.9,NaN,46.0
341,140000342,Rogerio,Nogueira,Minotouro,Light Heavyweight,1976-06-02,74.0,205.0,75.0,23.0,...,0.0,5.7,53.7,1.6,2.0,37.4,37.4,32.7,NaN,45.0
717,140000720,Fabricio,Werdum,Vai Cavalo,Heavyweight,1977-07-30,76.0,241.0,77.0,24.0,...,0.0,8.6,94.6,4.2,2.8,27.3,27.3,27.3,NaN,44.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,140001164,Islam,Mamedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1153,140001165,Brent,Primus,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,140001166,Usman,Nurmagomedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,140001167,Luis,Muro,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
active_fighters.sort_values(by = 'age', ascending = True)

,FighterId,FirstName,LastName,Nickname,WeightClass,BirthDate,Height,Weight,Reach,Wins,...,TitleDraws,CareerStats.SigStrikesLandedPerMinute,CareerStats.SigStrikeAccuracy,CareerStats.TakedownAverage,CareerStats.SubmissionAverage,CareerStats.KnockoutPercentage,CareerStats.TechnicalKnockoutPercentage,CareerStats.DecisionPercentage,CareerStats,age
964,140000970,Cory,McKenna,The Hobbit,Women's Strawweight,1999-11-07,63.0,115.0,58.0,6.0,...,0.0,8.3,69.2,1.2,0.0,0.0,0.0,22.1,NaN,21.0
1101,140001113,Rong,Zhu,NaN,Lightweight,2000-03-07,69.0,156.0,71.0,18.0,...,0.0,6.7,66.4,7.2,0.0,0.0,0.0,0.0,NaN,21.0
817,140000820,Chase,Hooper,The Dream,Featherweight,1999-09-13,73.0,145.0,74.0,10.0,...,0.0,9.2,90.0,2.3,5.1,11.9,11.9,11.9,NaN,22.0
926,140000930,Kay,Hansen,NaN,Women's Strawweight,1999-08-14,62.0,115.0,63.0,7.0,...,0.0,8.5,91.7,5.2,3.9,0.0,0.0,0.0,NaN,22.0
944,140000950,Luis,Rodriguez,Lazy Boy,Bantamweight,1999-05-07,66.0,125.0,65.5,11.0,...,0.0,3.0,57.5,4.8,4.8,0.0,0.0,0.0,NaN,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,140001164,Islam,Mamedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1153,140001165,Brent,Primus,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1154,140001166,Usman,Nurmagomedov,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,140001167,Luis,Muro,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
active_fighters["Fighter Name"] = active_fighters["FirstName"] + ' ' + active_fighters["LastName"]

In [54]:
active_fighters = active_fighters[["FighterId","Fighter Name", "age", "FirstName", "LastName", "Nickname",
                     "WeightClass", "BirthDate", "Height", "Weight", "Reach", "Wins", "Losses", "Draws", 
                     "NoContests", "TechnicalKnockouts", "TechnicalKnockoutLosses", "Submissions", 
                     "SubmissionLosses", "TitleWins", "TitleLosses", "TitleDraws", 
                     "CareerStats.SigStrikesLandedPerMinute", "CareerStats.SigStrikeAccuracy", 
                     "CareerStats.TakedownAverage", "CareerStats.SubmissionAverage",
                     "CareerStats.KnockoutPercentage", "CareerStats.TechnicalKnockoutPercentage", 
                     "CareerStats.DecisionPercentage"]]

In [55]:
total_fights = active_fighters["Wins"] + active_fighters["Losses"] + active_fighters["Draws"]

In [56]:
active_fighters["total_fights"] = total_fights

In [57]:
win_pct = ((active_fighters["Wins"] / active_fighters["total_fights"]) * 100)

In [58]:
active_fighters["win_pct"] = win_pct

In [59]:
title_fights = active_fighters["TitleWins"] + active_fighters["TitleLosses"] + active_fighters["TitleDraws"]

In [60]:
active_fighters["title_fights"] = title_fights

In [61]:
title_win_pct = ((active_fighters["TitleWins"] / active_fighters["title_fights"]) * 100)

In [62]:
active_fighters["title_win_pct"] = title_win_pct

In [63]:
active_fighters[['Fighter Name', 'TitleWins', 'TitleLosses', 'TitleDraws',
                 'total_fights', 'Wins', 'Losses', 'Draws']]

,Fighter Name,TitleWins,TitleLosses,TitleDraws,total_fights,Wins,Losses,Draws
0,Chance Rencountre,0.0,0.0,0.0,18.0,14.0,4.0,0.0
1,Kyle Stewart,0.0,2.0,0.0,16.0,13.0,3.0,0.0
2,Geoff Neal,0.0,0.0,0.0,17.0,13.0,4.0,0.0
3,Belal Muhammad,2.0,0.0,0.0,22.0,19.0,3.0,0.0
4,Dennis Bermudez,0.0,0.0,0.0,26.0,17.0,9.0,0.0
...,...,...,...,...,...,...,...,...
1158,Bruno Korea,0.0,0.0,0.0,16.0,12.0,3.0,1.0
1159,C.J. Vergara,0.0,0.0,0.0,11.0,8.0,2.0,1.0
1160,Javid Basharat,0.0,0.0,0.0,10.0,10.0,0.0,0.0
1161,Saimon Oliveira,0.0,0.0,0.0,20.0,17.0,3.0,0.0
